In [8]:
PORT = 10025

In [9]:
import socket
import sys
from joblib import Parallel, delayed
from math import sqrt

from socket import error as SocketError
import errno
import time




def RunRequest(k, responses, data, message, sock):
    # Create a TCP/IP socket
    # Connect the socket to the port where the server is listening
    server_address = ('localhost', PORT)
    try:
        sock[k].connect(server_address)
        # Send data
        message[k] = 'User: '+str(k)
        print('\n Sending "%s"' % message[k])
        sock[k].send( str.encode(message[k]) )

        # Look for the response
        amount_received = 0
        amount_expected = 1000000
        responses[k] = ''
        start=time.time()
        now=time.time()

        while (amount_received < amount_expected) and (start+4>now):
            data[k] = sock[k].recv(2014)
            amount_received += len(data[k].decode('utf-8'))
            responses[k] += data[k].decode('utf-8')
            now=time.time()
        print('************ Received "%s" ************ \n' % responses[k])
        sock[k].close()
        return

    except SocketError as e:
        if e.errno != errno.ECONNRESET:
            f=0
            #print("Error raro")
        else:
            f=0
            #print("Error reset")
    finally:
        sock[k].close()
        return
        #print("DONE")
    sock[k].close()
    return

n=15
responses=[]
data=[]
message=[]
sock=[]
for k in range(n):
	responses.append("")
	data.append("")
	message.append("")
	sock.append(socket.socket(socket.AF_INET, socket.SOCK_STREAM))
	
#Parallel(n_jobs=2, backend="threading")(delayed(sqrt)(i ** 2) for i in range(10))
Parallel(n_jobs=n, backend="threading")(delayed(RunRequest)(k, responses, data, message, sock) for k in range(n))




 Sending "User: 6"
 Sending "User: 0"
 Sending "User: 7"
 Sending "User: 5"
 Sending "User: 4"
 Sending "User: 1"
 Sending "User: 3"
 Sending "User: 9"
 Sending "User: 10"
 Sending "User: 8"
 Sending "User: 12"
 Sending "User: 13"
 Sending "User: 11"
 Sending "User: 2"
 Sending "User: 14"














************ Received "
SERVIDOR: INICIANDO (self.port  45568): User: 12
	------SERVIDOR: CORRIENDO (45568)
		SERVIDOR: TERMINANDO (45568): User: 12" ************ 
************ Received "
SERVIDOR: INICIANDO (self.port  45566): User: 1
	------SERVIDOR: CORRIENDO (45566)
		SERVIDOR: TERMINANDO (45566): User: 1" ************ 
************ Received "
SERVIDOR: INICIANDO (self.port  45554): User: 4
	------SERVIDOR: CORRIENDO (45554)
		SERVIDOR: TERMINANDO (45554): User: 4" ************ 
************ Received "
SERVIDOR: INICIANDO (self.port  45570): User: 13
	------SERVIDOR: CORRIENDO (45570)
		SERVIDOR: TERMINANDO (45570): User: 13" ************ 
************ Received "
SERVIDOR: INICIANDO

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]